This notebook demonstrates how ENCODE reproducibly processes raw files from ChIP-seq experiments using Docker, WDL, Cromwell, and AWS compute.

* Create AWS batch compute environment
* Create AWS config for Cromwell
* Find raw files of interest on ENCODE Portal
* Launch job using ENCODE files from S3
* Use boto3 to list resulting files